In [12]:
import pygame
import math
from pygame.draw import *
from pygame.event import *
from random import *
import time
pygame.init()

# start condition
score = 0
hight_screen = 700
width_screen = 1300
start_time = time.time()
time_game = 10
# balls
count_ball = 8
max_speed_ball = 15
min_speed_ball = 5
max_radius = 50
min_radius = 20
max_alpha_ball = 2 * math.pi
balls = []
# square
count_square = 3
max_speed_square = 15
min_speed_square = 10
max_side = 70
min_side = 20
squares = []


FPS = 30
screen = pygame.display.set_mode((width_screen, hight_screen))

colors = [(0, 0, 0), # black
          (173, 253, 47), # green_yellow
          (0, 255, 0), # lime
          (50, 205, 50), # lime_green
          (0, 128, 0), # green
          (0, 255, 127), # spring_green
          (0, 100, 0), # dark_green
          (154, 205, 50), # yellow_green
          (107, 142, 35), # olive_grab
          (124, 252, 0), # lawn_green
        ]

def new_ball():
    '''
    Draw new ball
    Return the parameters of the ball:
    ball[0] is radius of ball
    ball[1] is x coordinate of ball center
    ball[2] is y coordinate of ball center
    ball[3] is direction angle of ball
    ball[4] is speed of ball
    ball[5] is color of ball
    '''
    
    radius = randint(min_radius, max_radius)
    x_start_ball = randint(radius, width_screen - radius)
    y_start_ball = randint(radius, hight_screen - radius)
    alpha_ball = randint(0, int(max_alpha_ball))
    speed_ball = randint(min_speed_ball, max_speed_ball)
    color = colors[randint(1, 8)]
    circle(screen, color, (x_start_ball, y_start_ball), radius)
    return [radius, x_start_ball, y_start_ball, alpha_ball, speed_ball, color]


def new_square():
    '''
    Draw new square
    Return the parameters of the square:
      ball[0] is side of square
      ball[1] is x coordinate of square center
      ball[2] is y coordinate of square center
      ball[3] is speed of square
      ball[4] is color of square
    '''
    
    side = randint(min_side, max_side)
    x_start_square = randint(side // 2, width_screen - side // 2)
    y_start_square = randint(side // 2, hight_screen - side // 2)
    speed_square = randint(min_speed_square, max_speed_square)
    color = colors[randint(1, 8)]
    polygon(screen, color, ((x_start_square - side // 2, y_start_square - side // 2),
            (x_start_square + side // 2, y_start_square - side // 2),
            (x_start_square + side // 2, y_start_square + side // 2),
            (x_start_square - side // 2, y_start_square + side // 2))
            )
    return [side, x_start_square, y_start_square, speed_square, color]
    
    
def move_ball(ball: list):
    '''
    Moves the ball according to its paraveters
    Change the angle when reflected from the wall
    Gets parameters of one ball
    Returns new ball parameters
    '''
    
    color = ball[5]
    radius = ball[0]
    x = ball[1]
    y = ball[2]
    if x >= width_screen - radius:
        ball[3] = math.pi / 2 + random() * math.pi
    if y >= hight_screen - radius:
        ball[3] = random() * math.pi
    if radius >= x:
        ball[3] = random() * math.pi - math.pi / 2
    if radius >= y:
        ball[3] = random() * math.pi + math.pi
    speed = ball[4]
    alpha_ball = ball[3]
    x += int(speed * math.cos(alpha_ball))
    y -= int(speed * math.sin(alpha_ball))
    ball[1] = x
    ball[2] = y
    circle(screen, color, (x, y), radius)
    return ball


def move_square(square: list, squares: list):
    '''
    Reduces the size of the square to according to its parameters
    Goes to function delete_square at zero side of the sguare
    Gets parameters of one square and the whole list of squares
    Returns the modified square parameters and the whole list of squares
    '''
    
    color = square[4]
    side = square[0]
    x = square[1]
    y = square[2]
    speed = square[3]
    side -= 1 * int(speed * 0.2)
    if side <= 1:
        delete_square(square, squares)
    polygon(screen, color, ((x - side // 2, y - side // 2),
            (x + side // 2, y - side // 2),
            (x + side // 2, y + side // 2),
            (x - side // 2, y + side // 2))
           )
    square[0] = side
    return square, squares
    

def delete_ball(ball:list, balls:list):
    '''
    Replaces the "ball" in the "balls" list with a new one
    To do this, calls the function new_ball
    Gets parameters of one ball, which need to delete and the whole list of balls
    Returns the modified nested list "ball" in list "balls"
    '''
    
    balls[balls.index(ball)] = list(new_ball())
    return balls


def delete_square(square:list, squares:list):
    '''
    Replaces the "square" in the "squares" list with a new one
    To do this, calls the function new_square
    Gets parameters of one square, which need to delete and the whole list of squares
    Returns the modified nested list "square" in list "squares"
    '''
    
    squares[squares.index(square)] = list(new_square())
    return squares


def click_ball(ball: list, event, score: int, balls:list):
    '''
    Checks if the player hits the ball
    Calls funtion delete_ball when hitting the ball
    Gets the following parameters:
      ball is parameters of one ball
      score is current hit count
      balls is list of all balls
    Returns the modified nested list "ball" in list "balls" and "score"
    '''
    
    radius = ball[0]
    x_ball = ball[1]
    y_ball = ball[2]
    distance = math.sqrt((event.pos[0] - x_ball) ** 2 + (event.pos[1] - y_ball) ** 2)
    if distance <= radius:
        delete_ball(ball, balls)
        score = score + 1
    return balls, score


def click_square(square:list, event, score: int, squares:list):
    '''
    Checks if the player hits the square
    Calls funtion delete_square when hitting the square
    Gets the following parameters:
      square is parameters of one square
      score is current hit count
      squares is list of all squares
    Returns the modified nested list "square" in list "squares" and "score"
    '''
    
    side = square[0]
    x_square = square[1]
    y_square = square[2]
    if (event.pos[0] <= x_square + side // 2) and (event.pos[0] >= x_square - side // 2) and (event.pos[1] <= y_square + side // 2) and (event.pos[1] >= y_square - side // 2):
        delete_square(square, squares)
        score = score + 5
    return squares, score    


# name player
print("Enter yor name ---> ")
name = input()

# generetion of the first balls
for ball in range(count_ball):
    balls.append(list(new_ball()))

# generetion of the first sguares
for square in range(count_square):
    squares.append(list(new_square()))


clock = pygame.time.Clock()
finished = False

while not finished:
    clock.tick(FPS)
    if time.time() - start_time > time_game:
        finished = True
    pygame.display.update()
    screen.fill(colors[0])
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            finished = True
        elif event.type == pygame.MOUSEBUTTONDOWN:
            for ball in balls:
                balls, score = click_ball(ball, event, score, balls)
            for square in squares:
                squares, score = click_square(square, event, score, squares)
    for ball in balls:
        move_ball(ball)
    for square in squares:
        move_square(square, squares)
pygame.quit()

file = open('Score_table', 'r')

for line in file:
    player_score = line.split()[-1] # the result of one other player
    print(player_score)
    if score >= int(player_score):
        line_person = line + name + '  --->  ' + str(score)
        file = open('Score_table', 'a')
        file.write(line_person + "\n")
        file.close()
        break


print('Your score -> ', score)

Enter yor name ---> 
sdfdf
12
8
4
Your score ->  7
